<a href="https://colab.research.google.com/github/satoicu/aes-rsa-crypt/blob/main/AES%26RSA_Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RSA/AESを使うためpycryptodomeをインストールします

In [2]:
!pip install pycryptodome

これから仮のパスワードをAESを用いて暗号化、AES複合化に必要な情報をRSA暗号化します。

この処理によりRSA複合化するための公開鍵を持ったユーザーだけがAES複合化に必要な情報にアクセスできます。

まず初めに秘密鍵、公開鍵を作成します。

In [3]:
!ssh-keygen

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:uFgDlLikDVJGsa4dFslyA6KvRrbZBwhmWfoOhCHSRA8 root@d67a97a7d9a6
The key's randomart image is:
+---[RSA 3072]----+
|**E+..           |
|O=B*.            |
|*@B.o            |
|*==o . .         |
| ==o  + S        |
|o=*..o o         |
|o+.o...          |
|.   .            |
|                 |
+----[SHA256]-----+


秘密鍵（id_rsa）、公開鍵（id_rsa_pub）を作成しました。

パスフレーズは省略します。

パスフレーズを省略しない場合鍵情報の利用時に毎回パスフレーズの入力が必要になります。

サーバーやプログラムでの処理には向きません。

In [6]:
!cp /root/.ssh/id_rsa id_rsa
!cp /root/.ssh/id_rsa.pub id_rsa.pub
!ls

id_rsa	id_rsa.pub  sample_data


仮に暗号化したい文字列を「password」とします。

In [8]:
plaintext = "password"

「password」をAES暗号化します。

In [18]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = plaintext.encode('utf-8')

# 鍵の生成
key = get_random_bytes(32)

# AES暗号化
cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)
iv = cipher.nonce

ciphertext, key, tag, iv

(b"\xd0\x90ZQ\x86'\xed\x96",
 b'\x1a\xdc%N\xe2\xd9\xcb\x9az8\x87\x08\x8dj\xa1D\xc5\xfc\x82l\xed\xb5\xec\xf3$\xa1\x9c\x8e\x0co/s',
 b'4\xdee\x13\xef`\xde\xc8wq\x8d\x0f;\x98Cp',
 b'\xd4\x83\x81\x13-k\xf8\xde\xd1\x9e\x82\xcaJ\xd8\xf6\xec')

このままではAESで暗号化したkey, tag, ivはそのままであり容易に複合化できます。

そのためkey, tag, ivをさらにRSA暗号化します。

In [19]:
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA

#  keyinfoとtag, iv, keyの結合
keyinfo = key + b'@' + tag + b'@' + iv

# RSA暗号化のため公開鍵の読み込み
with open("id_rsa.pub", "rb") as f:
    public_key = RSA.import_key(f.read())

# 暗号化
rsa = PKCS1_OAEP.new(public_key)
cipher_keyinfo = rsa.encrypt(keyinfo)

cipher_keyinfo

b'\x18\xd0\x81\x15\x00\x94\xa5\x1e\xbe\xea(b\xe6\x8dlw\xab\xbdj\xc1\xc7,\x9a\x14\x06\xbd\x02O9\n\x0f\xf8\xb5>@\xed\xfe\xb4_\x0f\x93\x1e{\xfeF[\x10\xd8t\x02{]\x84\xe1\xdd\xaalu\xf7\x92\xe1\xfez\x87\x1b\xbc\xb2\xb3ht\x19\x98\xf5\xee6\xe3E(\\\xbd\xa3Mh\x0f\xc3\xb0\xb4z\xe1\x91:Z>\xcd\x88na\xe7\xd2?!P\xb4\xa3\x083\x16\xdd\xca\xd8\xbf>\x99\xc5\xa8\xf3?j\xe9\x90v\xd4h\'\x82\xfc\x19\x01\x8f\x8f\xa8\xa5\xb8H\xe1!\xeeP9\x1f\x90\x96\xc1I\xfaM"\xad!p\x1dF\xf0@\xbe\x92\x80\xba\x1a]I\xd3\xec\xdd\x0b\x14\xa9\x8c\xb2\x86S\xbd\x0e\xae\rnw\xfd\xfbi>\\A\x97\xd4#\x97\xf6\xed|\x99 \r\x07\' l\xe9\x16\xe4\x03\x8b>\xdb\x9d+\xa1[`\x0c\xca\xee\xf3\x15\xb7\x9c\x9a(\xc5_\xd9\xf4\\\xfcL\xa38X\x18e\x19U\\\x93\xfbB\xd9\xd6\xd4\xbd0\xd4\xb2\x99\xb7\x84\xa2,g\x91\\A\xda_\xa1\xfe\x80\x91$!\xbel\x86\xe4\x81\xcdS\xa6\x0cb&E\x9c\x88\xfcO\x12e\xfe\x8d\xc3\x03"\xdeu>\x1ds2\x02\xd7\xf1\xa2\x02\xd0\xfa#\x94\x81\xe58\x17\xbd{\xd2\xfe\xae:jN\xf9\xdf\\:\xec\x17\xad%_~^\xef<\xf4\xe9:5\xb26*\xcc\xf9\x80\xa2\x92r\x91r^4gw\xd6\x15\

key, tag, ivをkeyinfoというバイト文字列にまとめRSA暗号化しました。

複合には対応する秘密鍵が必要になります。

暗号化されたcipher_keyinfoは元のkeyinfoとは異なるものであることが確認できます。

In [20]:
# 暗号化されたため元の鍵情報とは異なりFalseが返されます。
cipher_keyinfo == keyinfo

False

元々のパスワード、keyinfoを全て削除します。これにより複合を行わないと元のパスワードはわからなくなります。

In [22]:
del plaintext, key, tag, iv, keyinfo

元のplaintextを知るためにまず秘密鍵を用いてkeyinfoを複合化、その後、keyinfoを用いてplaintextを複合化します。

まず、秘密鍵を読み込みkeyinfoを複合化します。

In [25]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA

with open("id_rsa", "rb") as f:
  private_key = RSA.import_key(f.read())

# RSA復号化
decipher_rsa = PKCS1_OAEP.new(private_key)
keyinfo = decipher_rsa.decrypt(cipher_keyinfo)

# key, tag, ivを取り出す
key, tag, iv = keyinfo.split(b'@')

key, tag, iv

(b'\x1a\xdc%N\xe2\xd9\xcb\x9az8\x87\x08\x8dj\xa1D\xc5\xfc\x82l\xed\xb5\xec\xf3$\xa1\x9c\x8e\x0co/s',
 b'4\xdee\x13\xef`\xde\xc8wq\x8d\x0f;\x98Cp',
 b'\xd4\x83\x81\x13-k\xf8\xde\xd1\x9e\x82\xcaJ\xd8\xf6\xec')

取り出した鍵情報を元にplaintextをAES複合化します。



In [26]:
# AES復号化
decipher_aes = AES.new(key, AES.MODE_EAX, iv)
decrypted_text = decipher_aes.decrypt_and_verify(ciphertext, tag)

# 複合化されたテキストの表示
plaintext = decrypted_text.decode("utf-8")
plaintext

'password'

In [33]:
# 暗号化・複合化処理を関数化
def encrypt(plaintext, pub_key="id_rsa.pub"):
  from Crypto.Cipher import PKCS1_OAEP
  from Crypto.PublicKey import RSA
  from Crypto.Cipher import AES
  from Crypto.Random import get_random_bytes

  data = plaintext.encode('utf-8')

  # 鍵の生成
  key = get_random_bytes(32)

  # AES暗号化
  cipher = AES.new(key, AES.MODE_EAX)
  ciphertext, tag = cipher.encrypt_and_digest(data)
  iv = cipher.nonce

  #  keyinfoとtag, iv, keyの結合
  keyinfo = key + b'@' + tag + b'@' + iv

  # RSA暗号化のため公開鍵の読み込み
  with open(pub_key, "rb") as f:
      public_key = RSA.import_key(f.read())

  # 暗号化
  rsa = PKCS1_OAEP.new(public_key)
  cipherkeyinfo = rsa.encrypt(keyinfo)

  return ciphertext, cipherkeyinfo

def decrypt(ciphertext, cipherkeyinfo, sec_key="id_rsa"):
  from Crypto.Cipher import AES, PKCS1_OAEP
  from Crypto.PublicKey import RSA

  with open(sec_key, "rb") as f:
    private_key = RSA.import_key(f.read())

  # RSA復号化
  decipher_rsa = PKCS1_OAEP.new(private_key)
  keyinfo = decipher_rsa.decrypt(cipherkeyinfo)

  # key, tag, ivを取り出す
  key, tag, iv = keyinfo.split(b'@')

  # AES復号化
  decipher_aes = AES.new(key, AES.MODE_EAX, iv)
  decrypted_text = decipher_aes.decrypt_and_verify(ciphertext, tag)

  # 複合化されたテキストの表示
  plaintext = decrypted_text.decode("utf-8")

  return plaintext

In [39]:
plaintext = "another password"
ciphertext, cipherkeyinfo = encrypt(plaintext)
decrypt(ciphertext, cipherkeyinfo)

'another password'

以上でAES、RSAを用いて特定の秘密鍵、公開鍵をやりとりした人にのみ暗号化した情報を伝えることができます。

次にプログラムで扱いやすくするためにバイト文字列をbase64を使って単なる文字列として取り扱います。

これによりjsonやyamlなどのテキストファイル形式で保存しやすくなります。
バイト文字列を単なる文字列に変換するにはbase64でエンコードした後utf-8でデコードします。

In [48]:
import base64

plaintext = "another text"
ciphertext, cipherkeyinfo = encrypt(plaintext)
b64ciphertext = base64.b64encode(ciphertext).decode('utf-8')
b64cipherkeyinfo = base64.b64encode(cipherkeyinfo).decode('utf-8')
print("base64 encoded ciphertext=", b64ciphertext)
print("base64 encoded cipherkeyinfo=", b64cipherkeyinfo)

base64 encoded ciphertext= CADWqpi70jA0zzmJ
base64 encoded cipherkeyinfo= C5O4tK9WnJaqllNBPijwzygZHsx7jiOq47VYUu7oX+XWJt2QmWBh1R2UD/ZG7/llSmmlvT9+HEJ468wGEDM694xr/qBhkHdAA6wYmguEKoG3ar11mOIgzmrWFZCVLhWpPpNLk4bNW66wH9RGzKvps0oTWRfQRotuyii6U0kbikcoucut0YKTQwH93C8I0S2EQdYbhv37zOLuk4sOMfa++8NolM2dvWtWgDpUCR4jir6VIbYSamolaE9woDMRlk0a7QYAG5ZWXOGQRabgVj6odACv+cdCYjjmJ4SwE254Aes37BrDad/aL4h0dW3AldOOou7H3dVF7M8pL2SSy3hncMX9E2yPatY1LPuaXihVtkVWA3ir7wNTfknIaDjH8zKbOkS8q+1QM/Y27ZLe9A7Dv/VMXVoFW/FetIYg9Av5BGKOC1mBEGq0DPmV/g+E98YWN7IPfBNA8xSlStm2uQYFfQutrZ2mXO6ckVtyppEANIqmlcJJ1RTiRSGyhKkSdGhh


utf-8のエンコードは文字列をバイト文字列に変換します。

utf-8のデコードはバイト文字列を単なる文字列に変換します。

basae64はバイト文字列を対応する64進法の英数字に当てはめて変換します。

In [49]:
b64decode_ciphertext = base64.b64decode(b64ciphertext)
b64decode_cipherkeyinfo = base64.b64decode(b64cipherkeyinfo)
decrypt(b64decode_ciphertext, b64decode_cipherkeyinfo)

'another text'

AESは非常に長い文字列も高速に暗号化できる特徴があります。

以下のような複数の情報をまとめたjson形式のファイルも暗号化できます。

サンプルはchat-gptを用いて生成した実在しない人物に対するデータの例です。

In [50]:
sample={
  "name": "John Doe",
  "age": 30,
  "email": "johndoe@example.com",
  "address": {
    "street": "123 Main St",
    "city": "New York",
    "state": "NY",
    "zip": "10001"
  },
  "phone_numbers": [
    "+1-555-1234",
    "+1-555-5678"
  ],
  "friends": [
    {
      "name": "Jane Smith",
      "age": 28,
      "email": "janesmith@example.com"
    },
    {
      "name": "Robert Johnson",
      "age": 32,
      "email": "robertjohnson@example.com"
    }
  ],
  "hobbies": [
    "reading",
    "playing guitar",
    "hiking"
  ],
  "description": "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum a quam auctor, faucibus quam in, sollicitudin nisi. Nulla in urna in elit tristique interdum ac at nulla. Proin euismod scelerisque sem vitae feugiat."
}

In [54]:
import json
message = json.dumps(sample)

ciphertext, cipherkeyinfo = encrypt(message)

In [55]:
decrypted_text = decrypt(ciphertext, cipherkeyinfo)

# 結果の表示
print("暗号化前のテキスト:", message)
print("復号化後のテキスト:", decrypted_text)
print(message == decrypted_text)

暗号化前のテキスト: {"name": "John Doe", "age": 30, "email": "johndoe@example.com", "address": {"street": "123 Main St", "city": "New York", "state": "NY", "zip": "10001"}, "phone_numbers": ["+1-555-1234", "+1-555-5678"], "friends": [{"name": "Jane Smith", "age": 28, "email": "janesmith@example.com"}, {"name": "Robert Johnson", "age": 32, "email": "robertjohnson@example.com"}], "hobbies": ["reading", "playing guitar", "hiking"], "description": "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum a quam auctor, faucibus quam in, sollicitudin nisi. Nulla in urna in elit tristique interdum ac at nulla. Proin euismod scelerisque sem vitae feugiat."}
復号化後のテキスト: {"name": "John Doe", "age": 30, "email": "johndoe@example.com", "address": {"street": "123 Main St", "city": "New York", "state": "NY", "zip": "10001"}, "phone_numbers": ["+1-555-1234", "+1-555-5678"], "friends": [{"name": "Jane Smith", "age": 28, "email": "janesmith@example.com"}, {"name": "Robert Johnson", "age": 32, "email"